In [2]:
pip install openai

  Using cached openai-1.64.0-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.64.0-py3-none-any.whl (472 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markitdown 0.0.1a4 requires puremagic, which is not installed.
markitdown 0.0.1a4 requires pydub, which is not installed.
python-pptx 1.0.2 requires XlsxWriter>=0.5.7, which is not installed.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.


In [15]:
import pickle

In [11]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
  # extra_headers={
  #   "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
  #   "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  # },
  model="nvidia/llama-3.1-nemotron-70b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ],
    temperature = 0,
    stream = True,
    max_tokens = 1024
    
    
)

for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end = '')

The question of the meaning of life is one of the most profound and elusive questions humanity has ever pondered. It touches upon various disciplines, including philosophy, religion, spirituality, and science, each offering different perspectives.

1. **Philosophical Perspectives**:
   - **Existentialism**: Suggests that life has no inherent meaning, and it's up to each individual to create their own purpose.
   - **Absurdism**: Posits that the human desire for meaning in a seemingly meaningless world is the root of absurdity, but in acknowledging this, we can find freedom to create our own meaning.
   - **Hedonism**: Argues that the meaning of life is to seek pleasure and avoid pain, though this can vary from Epicureanism (moderate pleasures) to more extreme forms.
   - **Stoicism**: Emphasizes living in accordance with reason and virtue, suggesting meaning is found in living a life of wisdom, self-control, and inner strength.

2. **Religious and Spiritual Perspectives**:
   - **Monot

In [79]:
def nemotron_llama(query, context):

    prompt_template = """
    you are an intelligent system who answers law related questions based on the provided context
    Instructions:
    - answer only from the context 
    - don't use your knowledge to answer the question
    - do not mention in your response that you refered to a external context for answering user's question
    - add citations to your answer like date and page number at the end of each sentences
    here is an example:
    user:What are the conditions under which a person of Indian origin residing outside India can be deemed a citizen of India?
    system:
    ## Conditions for Citizenship of Indian Origin Persons Residing Outside India

    A person of Indian origin residing outside India can be deemed a citizen of India if:
    
    1. **Birth Requirement**: They, either of their parents, or any of their grandparents were born in India as defined in the Government of India Act, 1935 (as originally enacted).
    2. **Residency Requirement**: They are ordinarily residing in a country outside India as defined in that Act.
    3. **Registration by Diplomatic/Consular Representative**: They have been registered as a citizen of India by a diplomatic or consular representative of India in the country where they are residing.
    4. **Application Requirement**: They must apply for such registration in the prescribed form and manner, as determined by the Government of the Dominion of India or the Government of India.

    Reference:
    **Title**:THE CONSTITUTION OF INDIA 
    page numbers:23,45,88

    - take the above as an example and follow the same structure
    - take the Page Number and Date from the context to cite your answer
    context:{}
"""
    prompt_template = prompt_template.format(context)
    client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key="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",
    )
    
    completion = client.chat.completions.create(
      # extra_headers={
      #   "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
      #   "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
      # },
      model="nvidia/llama-3.1-nemotron-70b-instruct:free",
      messages=[
          {"role":"system", "content":prompt_template},
          {"role": "user","content": query}
      ],
        temperature = 0,
        stream = True,
        max_tokens = 1024
        
        
    )
    
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end = '')

In [16]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('../../volumes/models/jina-embeddings-v2-base-en/', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['How is the weather today?', 'What is the current weather like today?'])
print(cos_sim(embeddings[0], embeddings[1]))

0.9341315


In [17]:
def get_embeddings(text:list):
    embeddings = model.encode(text)
    normalized_embeddings = embeddings/norm(embeddings[0])
    return normalized_embeddings

In [18]:
from retriever import vector_db_retriever

Index loaded successfully!
Number of vectors in the index: 404


In [19]:
with open("../../volumes/metadata/metadata.pkl", "rb") as p:
    metadata = pickle.load(p)

In [26]:
ids = list(metadata.keys())

In [20]:
query = " explain Seventh Amendment Act"
query_embeddings = model.encode([query])

In [21]:
result = vector_db_retriever(query_embeddings, 15)

In [25]:
result[0][0]

132

In [80]:
def rag(query):
    query_embeddings = model.encode([query])
    result = vector_db_retriever(query_embeddings, 10)
    indexes = result[0][0]
    context = ""
    for idx in indexes:
        hash_id = ids[idx]
        retrieved_results = metadata[hash_id]
        context+="Title:"+retrieved_results['title']+"\n"+"Date:"+retrieved_results['date']+"\n"+"Page Number:"+str(retrieved_results['page_no'])+"\n"+"Corpus:"+retrieved_results['text']+"\n\n"
    nemotron_llama(query, context)
    print()
    print("CONTEXT:")
    print(context)

In [81]:
rag(query="explain Seventh Amendment Act")

**Seventh Amendment Act, 1956: Key Provisions and Impacts**

**Date:** November 1, 1956 (w.e.f.) [1]
**Page Numbers:** Various (referenced from THE CONSTITUTION OF INDIA 2022, May 2022)
**Relevant Sections:**
- **Article 15** (not directly provided in the given context, but amendments mentioned)
- **Article 16** (amendments mentioned in the context, Page Number: 39)
- **Article 224** (amended, Page Number: 133)
- **Article 368** (not directly provided, but related to amendment process)
- **Seventh Schedule** (amendments in List I and List II, Page Numbers: 347, 349)

**Key Provisions and Impacts of the Seventh Amendment Act, 1956:**

1. **Reorganization of States**: The most significant aspect of this amendment was the reorganization of states on a linguistic basis, leading to the merger of some states and the creation of new ones. This was achieved through changes in the First Schedule of the Constitution. [2]

2. **Amendments to Article 16**:
   - **Residence Requirement**: The amend